In [1]:
import pandas as pd

In [2]:
# data reading from file
dataset = pd.read_csv(r"./balloons_dataset.csv")

In [3]:
dataset.head(5)

,color,size,act,age,inflated
0,YELLOW,SMALL,STRETCH,ADULT,T
1,YELLOW,SMALL,STRETCH,ADULT,T
2,YELLOW,SMALL,STRETCH,ADULT,T
3,YELLOW,SMALL,STRETCH,ADULT,T
4,YELLOW,SMALL,STRETCH,ADULT,T


In [4]:
# define the class predition
features = dataset.columns
features = features.drop('inflated')
features = list(features)
features

['color', 'size', 'act', 'age']

In [5]:
def cPString(trainData, featureName):
#     make unique set of class feature
    avcSet = dict()
    DistFeature = set(trainData[featureName])
    c1=0
    c2=0
    for val in trainData.inflated:
        if val=='T':
            c1=c1+1
        else:
            c2 = c2+1
    
#     define probability of class feature
    avcSet['T'] = c1/len(trainData.inflated)
    avcSet['F'] = c2/len(trainData.inflated)
    
    for x in DistFeature:
        avcSet[x+'|T']=0
        avcSet[x+'|F']=0
    indx=0    
    for feat in trainData[featureName]:
        avcSet[feat+'|'+trainData.inflated[indx]]+=1
        indx+=1
    
#     laplacian transform checking
    c1_error, c2_error = False, False
    for key in avcSet:
        if key[len(key)-1] == 'T' and avcSet[key] == 0:
            c1_error = True
        elif key[len(key)-1] == 'F' and avcSet[key] == 0:
            c2_error = True
            
#     calculate probability
    for key in avcSet:
        if key[len(key)-1] == 'T':
            if c1_error:
                avcSet[key] = (avcSet[key]+1) / (c1+len(DistFeature))
            else:
                avcSet[key] = avcSet[key] / c1
        else:
            if c2_error:
                avcSet[key] = (avcSet[key]+1) / (c2+len(DistFeature))
            else:
                avcSet[key] = avcSet[key] / c2
    return avcSet
    

In [6]:
def ConditionalProbability(data, features, classfeature):
    # define train dataset
    x_train = dataset
    dataStorage = dict()
    for feature in features:
        dataStorage.update(cPString(data[[feature, classfeature]], feature))
    return dataStorage

In [7]:
# calculate conditional probability
processedData = ConditionalProbability(dataset, features, 'inflated')

print(processedData)

def predict(frature1, frature2, frature3, frature4):
    probTrue = processedData[frature1+"|T"] * processedData[frature2+"|T"] * processedData[frature3+"|T"] * processedData[frature4+"|T"] * processedData["T"]
    probFalse = processedData[frature1+"|F"] * processedData[frature2+"|F"] * processedData[frature3+"|F"] * processedData[frature4+"|F"] * processedData["F"]
    percTrue = probTrue / (probTrue + probFalse)
    percFalse = probFalse / (probTrue + probFalse)
    if percTrue > percFalse:
        print("Inflated = True.\nProbability: "+str(percTrue))
    else:
        print("Inflated = False.\nProbability: "+str(percFalse))

{'T': 0.03333333333333333, 'F': 0.01, 'PURPLE|T': 0.5, 'PURPLE|F': 0.5, 'YELLOW|T': 0.5, 'YELLOW|F': 0.5, 'LARGE|T': 0.5, 'LARGE|F': 0.5, 'SMALL|T': 0.5, 'SMALL|F': 0.5, 'STRETCH|T': 0.9761904761904762, 'STRETCH|F': 0.3333333333333333, 'DIP|T': 0.023809523809523808, 'DIP|F': 0.6666666666666666, 'ADULT|T': 0.9761904761904762, 'ADULT|F': 0.3333333333333333, 'CHILD|T': 0.023809523809523808, 'CHILD|F': 0.6666666666666666}


In [8]:
predict('YELLOW', 'SMALL', 'STRETCH', 'ADULT')
predict('PURPLE', 'LARGE', 'DIP', 'CHILD')

Inflated = True.
Probability: 0.9662030118404414
Inflated = False.
Probability: 0.995766299745978
